In [1]:
import numpy as np
from initLayers import initLayers
from train import train
from rna import rna
from utils import normalize, splitData, evaluate_multiclass_with_one_hot

Carrega dados do dataset

In [2]:
DATASET_PATH = 'penguins.csv'
data = np.genfromtxt(DATASET_PATH, delimiter=',', skip_header=1)

Separa features e labels

In [3]:
features = data[:, :-3]
labels = data[:, -3:]

Separa dados para treinamento e para validação

In [4]:
featuresTrain, featuresVal, labelsTrain, labelsVal = splitData(features, labels)

Normaliza features

In [5]:
normalized_features, feature_scaler = normalize(featuresTrain)

Combina features e labels e formata observações para treinamento

Define a estrutura da rede, funções de ativação, funções de custo, épocas e learning rate

In [6]:
# Obtém a quantidade de neurons da camada de input
INPUT_NEURONS = len(normalized_features[0])

# Define a estrutura da rede e funções de ativação
layers = [
    (INPUT_NEURONS, 'RELU'),
    #(INPUT_NEURONS, 'RELU'),
    (3, 'SOFTMAX')
]

# Definição função de custo, épocas e learning rate
COSTF = "CATEGORICAL_ENTROPY"
EPOCHS = 50
LEARNING_RATE = 0.001

Inicializa os pesos e as funções das camadas

In [7]:
layers = initLayers(layers, INPUT_NEURONS)

Dispara o treinamento da rede

In [8]:
trainedParams = train(EPOCHS, LEARNING_RATE, layers, normalized_features, labelsTrain, COSTF)

Normaliza as features para validação

In [9]:
test_normalized_features = feature_scaler.transform(featuresVal)


Testa a rede treinada

In [10]:
predictions = []
for i, observation in enumerate(test_normalized_features):

  input = observation
  prediction = rna(input, trainedParams)

  one_hot_prediction = np.zeros_like(prediction)
  one_hot_prediction[np.argmax(prediction)] = 1

  predictions.append(one_hot_prediction)

Obtém métricas de avaliação

In [11]:
metrics = evaluate_multiclass_with_one_hot(labelsVal, predictions)

Evaluation Metrics:
accuracy: 0.9855
precision: 0.9860
